<a href="https://colab.research.google.com/github/hannape/Gruba-kreska/blob/main/Funkcja_reprezentacje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def funkcja_reprezentacje(path_wav, data_settype, representation_type, repr_1d_summary, summary_1d, sr, chunk_length_ms, \
                          n_fft, win_length, hop_length, window, f_min, f_max, n_mels, N, step, Np, K, tm, flock, tlock):
   
  '''
  Jest to funkcja zawierająca cały potrzebny kod z pliku 'reprezentacja śmieci/Untitled1', bez zbędnych printów, komentarzy itd.

  Args: 
    te wszystkie parametry wejściowe co powyżej, opisane w miarę w mainie

  Returns:
    file_names_train_set - nazwy nagrań wchodzące w skład zbioru
    ind_for_train_set - indeksy chunksów wybrane z poszczególnych nagrań
    info_chunksy - informacje o wybranych chunksach - te dane co wcześniej w dataframe, ale tylko dla wybranych chunksów 
      ['chunk_ids', 'chunk_start', 'chunk_end', 'has_bird', 'chunks_species', 'call_id', 'has_unknown', 'has_noise']
    representation_type - - to co na wejściu było, jaki jeden z pięciu typów reprezentacji
    repr_full  - wybrana reprezentacja dla każdego z wybranych chunksów z nagrań. Rozmiary:
      spektrogram: 1 nagranie - chunksy x 63 x 148
      mel-spektrogram: 1 nagranie - chunksy x 60 x 148
      multitaper: 1 nagranie - chunksy x 64 x 149
      8_classic: 1 nagranie - chunksy x 1-4 x 8
      8_classic_plus_MIR: 1 nagranie - chunksy x 1-4 x 39

  ''' 
  ## IMPORTY

  import os
  import contextlib
  import numpy as np
  import wave
  import math
  import pandas as pd
  import random
  from timeit import default_timer as timer
  from scipy.stats.mstats import gmean
  from statistics import mean
  from matplotlib import cm
  import librosa
  import matplotlib.pyplot as plt
  import libtfr
  import time

  
  # nie powinno mieć to wszystko train w nazwie, no ale już trudno. Chodzi o dowolny dataset który będzie na wejściu 
  # z funkcji wczytanie:
  file_names_train_set = data_settype[0]
  ind_for_train_set = data_settype[1]
  result_dataframe_train = data_settype[2]  # trzeba powyciągać tylko interesujące nas chunksy
  df_to_np = result_dataframe_train.to_numpy()  # bo nie umiem sobie poradiś jak jest to df, wiele funkcji które chcę użyć nie działa

  repr_full = [[] for _ in range(np.shape(file_names_train_set)[0])]
  info_chunksy = [[[] for _ in range(8)] for _ in range(np.shape(file_names_train_set)[0])]  # 8 kolumn w dataframie było
  indices = data_settype[1]
  print(file_names_train_set) 

  # k - kolejne nagrania w zbiorze 
  for k in range(0,np.shape(file_names_train_set)[0]): #np.shape(file_names_train_set)[0]):  # 87 dla zbioru train

    ##### pętla do wycięgnięcia info o interesujących chunksach
    empty_list = ([[] for _ in range(np.shape(indices[k])[0])])
    # warunki: bo może się zdarzyć że macierz tablica pustych elementów będzie w tych dwóch 'chunks_species', 'call_id', i wtedy np.take nie działa 
    if not any(df_to_np[k][4]) and not any(df_to_np[k][5]):  
      info_chunksy[k] = [np.take(df_to_np[k][0],indices[k]), np.take(df_to_np[k][1],indices[k]), np.take(df_to_np[k][2],indices[k]), np.take(df_to_np[k][3],indices[k]), \
                        empty_list, empty_list, np.take(df_to_np[k][6],indices[k]), np.take(df_to_np[k][7],indices[k])]
      print(np.shape(info_chunksy[k][5]), 'no calls at all')
    elif not any(df_to_np[k][5]):
      info_chunksy[k] = [np.take(df_to_np[k][0],indices[k]), np.take(df_to_np[k][1],indices[k]), np.take(df_to_np[k][2],indices[k]), np.take(df_to_np[k][3],indices[k]), \
                        np.take(df_to_np[k][4],indices[k]).tolist(), empty_list, np.take(df_to_np[k][6],indices[k]), np.take(df_to_np[k][7],indices[k])]
      print(np.shape(info_chunksy[k][5]), 'no calls of interest')
    else:
      info_chunksy[k] = [np.take(df_to_np[k][0],indices[k]), np.take(df_to_np[k][1],indices[k]), np.take(df_to_np[k][2],indices[k]), np.take(df_to_np[k][3],indices[k]), \
                        np.take(df_to_np[k][4],indices[k]).tolist(), np.take(df_to_np[k][5],indices[k]).tolist(), np.take(df_to_np[k][6],indices[k]), np.take(df_to_np[k][7],indices[k])]  

    start_time = time.time()
    
    repr = [[] for _ in range(np.shape(ind_for_train_set[k])[0])]
    #print(path_wav, file_names_train_set[k])
    #print(ind_for_train_set[k])
    ##### a to już pętla do reprezentacji, pojedyncze chunksy
    for num, i in enumerate(ind_for_train_set[k]):                        # 262 dla pierwszego nagrania w zbiorze train, bo tyle mamy tam chunksów
      
      # wczytanie chunka naszego półsekundowego:
      y, sr = librosa.load(path_wav + file_names_train_set[k], sr = 44100, offset = result_dataframe_train['chunk_start'][k][i]/sr, duration = chunk_length_ms/1000)
      
      ####### W zależności od reprezentacji: 
      
      ####### reprezentacja 1  ------- 63 x 148 ------ SPEKTROGRAM
      if representation_type=='spektrogram':
      
        stft = librosa.stft(y, n_fft=n_fft, win_length=win_length, hop_length=hop_length, window= window) # krótkoczasowa transformata fouriera STFT
        stft1 = librosa.amplitude_to_db(np.abs(stft)**2)                                              # kwadrat wartości bezwzględnej plus przerzucenie na skalę decybelową
        freqs = librosa.core.fft_frequencies(n_fft=n_fft, sr=sr)                             # wyznaczenie częstotliwości
        x,  = np.where( freqs >= min(freqs[(freqs >= f_min)]))
        j,  = np.where( freqs <= max(freqs[(freqs <= f_max)]))
        stft1 = stft1[min(x):max(j),]                                                        # stft tylko w wybranym zakresie
        #freqs1 = freqs[min(x):max(j),]
        repr1_spectro = stft1
        repr[num] = repr1_spectro

      ####### reprezentacja 2 (3 V3) - mel spektrogram  ------- 60 x 148 ------  
      if representation_type=='mel-spektrogram':
        # to chyba wzięte było z birdvoxa

        stft = librosa.stft(np.array(y), n_fft=n_fft, win_length=win_length, hop_length= hop_length, window=window)
        abs2_stft = (stft.real*stft.real) + (stft.imag*stft.imag)
        melspect = librosa.feature.melspectrogram(y=None, S=abs2_stft, sr=sr, n_mels= n_mels, fmin = f_min, fmax=f_max, hop_length=hop_length, n_fft=n_fft)
        repr2_melspec = 0.5 * librosa.amplitude_to_db(melspect, ref=1.0)
        repr[num] = repr2_melspec

      ####### reprezentacja 3 (5b) - multitaper o większej rozdzielczości ------- 64 x 149 ------
      # Ale tak naprawdę to nie jest multitaper, ale coś innego, nie wiem w sumie do końca co. Time-frequency reassignement spectrogram? DOCZYTAĆ
      if representation_type=='multitaper':
        result5b = libtfr.tfr_spec(y, N = N, step = step, Np = Np, K = 2, tm = tm, flock = flock, tlock = tlock)     
        freqs, ind = libtfr.fgrid(sr, N, fpass=(f_min,f_max)) 
        repr3_multitaper = result5b[ind,]; # tylko interesujące nas pasmo
        repr[num] = repr3_multitaper

      if representation_type=='8_classic' or representation_type=='8_classic_plus_MIR':

        S1 = np.abs(librosa.stft(np.array(y), n_fft=n_fft, win_length=win_length, hop_length= hop_length, window=window))
        freqs = librosa.core.fft_frequencies(n_fft=n_fft, sr=sr) 
        o,  = np.where( freqs >= min(freqs[(freqs >= f_min)]))
        j,  = np.where( freqs <= max(freqs[(freqs <= f_max)]))
        freqs1 = freqs[min(o):max(j),]
        S = S1[min(o):max(j),] 

        param_0 = np.sum(S, axis=0)             #librosa.feature.spectral_bandwidth(S=S, p = 1)  # moc sygnału. Ale z wartości absolutnej spektorgramu, bez tego kwadratu jeszcz
        param_1 = librosa.feature.spectral_centroid(S=S, freq=freqs1)                            # centroid https://www.mathworks.com/help/audio/ref/spectralcentroid.html
        param_2 = np.power(librosa.feature.spectral_bandwidth(S=S, freq=freqs1, p = 2),2)        # 2 rzędu  
        param_3 = np.power(librosa.feature.spectral_bandwidth(S=S, freq=freqs1, p = 3),3)        # 3 rzędu
        param_4 = np.power(librosa.feature.spectral_bandwidth(S=S, freq=freqs1, p = 4),4)        # 4 rzędu
        skosnosc = param_3[0] / np.power(param_2[0],1.5)                        # https://www.mathworks.com/help/audio/ref/spectralskewness.html #skosnosc2 = skew(S, axis=0)
        kurtoza =  param_4[0]/ np.power(param_2[0],2) - 3                       #kurtoza2 = kurtosis(S, axis=0)
        plaskosc = librosa.feature.spectral_flatness(S=S)                       #gmean(S_squared)/np.mean(S_squared)
        nb_summary = np.sum(summary_1d)
                
        if representation_type=='8_classic_plus_MIR':
 
          def FeatureSpectralFlux(X):  #https://www.audiocontentanalysis.org/code/audio-features/spectral-flux-2/
            # difference spectrum (set first diff to zero)
            X = np.c_[X[:, 0], X]
            afDeltaX = np.diff(X, 1, axis=1)
            # flux
            vsf = np.sqrt((afDeltaX**2).sum(axis=0)) / X.shape[0]
            return  vsf[1:]                 # pozbycie się pierwszego elementu, który zawsze jest zerem , np.squeeze(vsf[1:]) if isSpectrum else
          
          stft = librosa.stft(np.array(y), n_fft=n_fft, win_length=win_length, hop_length= hop_length, window=window)
          abs2_stft = (stft.real*stft.real) + (stft.imag*stft.imag)
          melspect = librosa.feature.melspectrogram(y=None, S=abs2_stft, sr=sr, n_mels= n_mels, fmin = f_min, fmax=f_max, hop_length=hop_length, n_fft=n_fft)
          
          mfccs =librosa.feature.mfcc(S=librosa.power_to_db(melspect), n_mfcc=12)
          mfcc_delta = librosa.feature.delta(mfccs)
          zcr = sum(librosa.feature.zero_crossing_rate(y, frame_length=win_length, hop_length= hop_length))  #  ZCR can be interpreted as a measure of the noisiness of a signal. For example, it usually exhibits higher values in the case of noisy signals. It is also known to reflect, in a rather coarse manner, the spectral characteristics of a signal. std to recognize speech vs music
          contrast = librosa.feature.spectral_contrast(S=S, hop_length=hop_length, n_fft=n_fft, n_bands = 2, fmin = f_min)
          rolloff = librosa.feature.spectral_rolloff(S=S, hop_length=hop_length, n_fft=n_fft, roll_percent=0.85)    
          rms = librosa.feature.rms(S=S, hop_length=hop_length, frame_length = 124) 
          spectral_flux = FeatureSpectralFlux(S)
          np.shape(rms)
          paramsy = [[[] for _ in range(39)] for _ in range(nb_summary)]
          idx = 0

          for m in range(np.shape(summary_1d)[0]):
            
            if summary_1d[m]:
              f = getattr(np, repr_1d_summary[m])  # która statystyka wybrana repr_1d_summary = ['min', 'max', 'mean', 'std']
              paramsy_mir = [f(param_0), f(param_1), f(param_2), f(param_3), f(param_4), f(skosnosc), f(kurtoza), f(plaskosc)]
              paramsy_mir.extend(f(mfccs, axis = 1).tolist())
              paramsy_mir.extend(f(mfcc_delta, axis = 1).tolist())
              paramsy_mir.extend([f(zcr)])
              paramsy_mir.extend(f(contrast, axis = 1).tolist())
              paramsy_mir.extend([f(rolloff), f(rms), f(spectral_flux)])
              paramsy[idx]= paramsy_mir
              idx += 1

          repr[num] = paramsy
        
        if representation_type=='8_classic':

          paramsy = [[[] for _ in range(8)] for _ in range(nb_summary)]
          idx = 0

          for m in range(np.shape(summary_1d)[0]):
            
            if summary_1d[m]:
              f = getattr(np, repr_1d_summary[m])
              paramsy[idx]=[f(param_0), f(param_1), f(param_2), f(param_3), f(param_4), f(skosnosc), f(kurtoza), f(plaskosc)]
              idx += 1

          repr[num] = paramsy

    repr_full[k] = repr
    print(k,'-', file_names_train_set[k], '- chunks:', np.shape(ind_for_train_set[k])[0], '- time:', time.time()-start_time)

  return [file_names_train_set , ind_for_train_set, info_chunksy, representation_type, repr_full]     
    